In [17]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import numpy as np
import re
import string
import nltk

from wordcloud import WordCloud, STOPWORDS
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv('data/book_review_labelled_data.csv')
print(train_df.shape)
train_df = train_df.drop('overall', axis=1)
train_df.head(10)

(49992, 9)


,reviewerID,reviewerName,reviewText,summary,reviewTime,rates_count,helpful_count,rating
0,A3UPFTGAWZ3G2R,David J. Loftus,"Jenkins, a history professor and Member of Par...","Quite readable, nicely done","12 6, 2001",40,37,4
1,A1XTKTLNSCRLDS,Ellen Rappaport,Detective Inspector Erlendur Sveinsson is at h...,Mesmerizing in depth,"02 23, 2014",0,0,5
2,A1A77B6DQQH436,"crescamp ""esc""",I didn't read this. I purchased it for a gift...,10-minute life lessons for kids,"02 12, 2013",3,0,3
3,AEAF4MRYHJZI,"Angelia Menchan ""acvermen.blogspot.com""",Fierce Angels by Sheri Park reads like a disse...,So FIERCE,"03 24, 2010",9,9,4
4,A3B7KU72LGWFER,"Grifel ""Tea Time""",Clearly this author had two goals in mind: 1) ...,Drivel!,"06 21, 2003",19,13,1
5,A3JD07VHDLT5FF,"isala ""Isabel and Lars""",This is a collection of stories and memories b...,Compelling stories by ordinary people,"03 19, 2005",7,5,5
6,A1JBVAMC9YU0WD,ED,I am so glad I borrowed this 'book' from the l...,this 'cheese' stinks,"09 25, 2000",6,5,1
7,A1O7OXC13G9X3E,"kayp75 ""kayp75""","Really enjoyable, couldn't put it down. The ch...",great read,"10 21, 2013",0,0,5
8,A1RAUVCWYHTQI4,A. Ross,I don't read a lot of science-fiction (maybe f...,Lots of Good Elements Fail to Add Up to an Eng...,"08 4, 2010",3,2,3
9,A3PZTH1DTX6O6B,"H. F. Gibbard ""History Buff""",Legendary pornographer Larry Flynt teamed up w...,fun but occasionally debatable history of pres...,"06 11, 2011",5,4,4


In [3]:
test_df = pd.read_csv('data/book_review_test_data_unlabelled.csv')
print(test_df.shape)
test_df.head(10)

(2000, 6)


,reviewerID,reviewerName,reviewText,summary,reviewTime,rating
0,A2HESNQJZ9OB7H,Jen,So boring and stupid had a hard time finishing...,Unbelievable.,"02 16, 2014",1
1,A1ABXPSFA9PC8N,Ben Parker,Ill be the first to admit i'm not the best coo...,Easy and Clear Cooking,"11 7, 2012",5
2,AYVW3O6W8S5S4,Johnny in Texas,Doesn't tell you how to do anything... just s...,not bad,"02 25, 2014",3
3,A17GOTFSUAWN17,Tyson,Whenever I met my last two girlfriends in pers...,"Short book, but good primer on how to text girls","03 15, 2014",4
4,A2VO8K861AV83R,"Avatheps ""Avatheps""",I read the reviews and decided to take a chanc...,Very disappointing. I could not finish it.,"12 30, 2013",2
5,A60C622DQ0QNU,Eric Lass,This book is packed with some great tips on de...,Very Useful and Some Great Tips,"10 8, 2012",4
6,A1ULRFNOXE2I08,Kathy,"Thought this was a great read, but a little sh...",Kathy,"03 17, 2013",4
7,A31MN9X90I4SB6,"Jacqueline R Walton ""Jacqueline R. Walton""",I am sorry but I was bored with this book. I c...,Couldn't Finish,"10 10, 2013",1
8,A3BLT4A2WSSK6M,Barry Melius,I don't read to stimulate my mind but to relax...,Good Hearted Romp,"12 1, 2012",4
9,A313BU9OG6ZVPD,"Patricia ""pstub""",I tried to get into this book but just couldn'...,Nope,"05 3, 2014",1


In [33]:
train_df['useful_criteria'] = train_df['helpful_count']/train_df['rates_count']
print(train_df.shape)
train_df = train_df.dropna()
print(train_df.shape)

(28423, 15)
(28423, 15)


In [5]:
train_df['label_aux'] = train_df['useful_criteria'].apply(lambda x: 'true' if x >= 1 else 'false')

In [6]:
train_df.head()

,reviewerID,reviewerName,reviewText,summary,reviewTime,rates_count,helpful_count,rating,useful_criteria,label_aux
0,A3UPFTGAWZ3G2R,David J. Loftus,"Jenkins, a history professor and Member of Par...","Quite readable, nicely done","12 6, 2001",40,37,4,0.925000,false
1,A1XTKTLNSCRLDS,Ellen Rappaport,Detective Inspector Erlendur Sveinsson is at h...,Mesmerizing in depth,"02 23, 2014",0,0,5,NaN,false
2,A1A77B6DQQH436,"crescamp ""esc""",I didn't read this. I purchased it for a gift...,10-minute life lessons for kids,"02 12, 2013",3,0,3,0.000000,false
3,AEAF4MRYHJZI,"Angelia Menchan ""acvermen.blogspot.com""",Fierce Angels by Sheri Park reads like a disse...,So FIERCE,"03 24, 2010",9,9,4,1.000000,true
4,A3B7KU72LGWFER,"Grifel ""Tea Time""",Clearly this author had two goals in mind: 1) ...,Drivel!,"06 21, 2003",19,13,1,0.684211,false


In [ ]:
cleanup_re = re.compile('[^a-z]+')

def cleanup(sentence):
    sentence = sentence.lower()
    sentence = cleanup_re.sub(' ', sentence).strip()
#     sentence = " ".join(nltk.word_tokenize(sentence))
    return sentence

In [ ]:
train_df['reviewText_cleaned'] = train_df['reviewText'].apply(cleanup)

In [ ]:
stopwords = set(STOPWORDS)

#mpl.rcParams['figure.figsize']=(8.0,6.0)    #(6.0,4.0)
mpl.rcParams['font.size']=12                #10 
mpl.rcParams['savefig.dpi']=100             #72 
mpl.rcParams['figure.subplot.bottom']=.1 


def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))
    
    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()
    
show_wordcloud(train_df[train_df['label_aux']=='true']["reviewText"])

In [ ]:
show_wordcloud(train_df[train_df['label_aux']=='false']["reviewText"])

In [ ]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

In [ ]:
train_df['reviewText_tokenized'] = train_df['reviewText'].apply(lambda x: ' '.join(tokenizer.tokenize(x)))

In [ ]:
show_wordcloud(train_df[train_df['label_aux']=='false']['reviewText_tokenized'].values)

In [ ]:
show_wordcloud(train_df[train_df['label_aux']=='true']['reviewText_tokenized'].values)

# Script to label reviews

In [ ]:
null_df = train_df[train_df['useful_criteria'].isna()]

In [ ]:
for i, row in null_df.iterrows():
    print(row['summary'])
    print('\n')
    print(row['reviewText'])
    true = input()
    print('---------')
    if true == 'y':
        row['useful_criteria'] = 1
    elif true == 'n':
        row['useful_criteria'] = 0
    elif true == 'break':
        break
    else:
        continue
    null_df.at[i,'useful_criteria'] = row['useful_criteria']

In [ ]:
null_df.head()

In [ ]:
sample = pd.read_csv('data/sample_submission.csv')
sample

In [ ]:
train_df.head()

# Extra features

In [41]:
train_df = pd.read_csv('data/book_review_labelled_data.csv')
print(train_df.shape)
train_df = train_df.drop('overall', axis=1)
train_df.head(10)

(49992, 9)


,reviewerID,reviewerName,reviewText,summary,reviewTime,rates_count,helpful_count,rating
0,A3UPFTGAWZ3G2R,David J. Loftus,"Jenkins, a history professor and Member of Par...","Quite readable, nicely done","12 6, 2001",40,37,4
1,A1XTKTLNSCRLDS,Ellen Rappaport,Detective Inspector Erlendur Sveinsson is at h...,Mesmerizing in depth,"02 23, 2014",0,0,5
2,A1A77B6DQQH436,"crescamp ""esc""",I didn't read this. I purchased it for a gift...,10-minute life lessons for kids,"02 12, 2013",3,0,3
3,AEAF4MRYHJZI,"Angelia Menchan ""acvermen.blogspot.com""",Fierce Angels by Sheri Park reads like a disse...,So FIERCE,"03 24, 2010",9,9,4
4,A3B7KU72LGWFER,"Grifel ""Tea Time""",Clearly this author had two goals in mind: 1) ...,Drivel!,"06 21, 2003",19,13,1
5,A3JD07VHDLT5FF,"isala ""Isabel and Lars""",This is a collection of stories and memories b...,Compelling stories by ordinary people,"03 19, 2005",7,5,5
6,A1JBVAMC9YU0WD,ED,I am so glad I borrowed this 'book' from the l...,this 'cheese' stinks,"09 25, 2000",6,5,1
7,A1O7OXC13G9X3E,"kayp75 ""kayp75""","Really enjoyable, couldn't put it down. The ch...",great read,"10 21, 2013",0,0,5
8,A1RAUVCWYHTQI4,A. Ross,I don't read a lot of science-fiction (maybe f...,Lots of Good Elements Fail to Add Up to an Eng...,"08 4, 2010",3,2,3
9,A3PZTH1DTX6O6B,"H. F. Gibbard ""History Buff""",Legendary pornographer Larry Flynt teamed up w...,fun but occasionally debatable history of pres...,"06 11, 2011",5,4,4


In [42]:
train_df['useful_criteria'] = train_df['helpful_count']/train_df['rates_count']
print(train_df.shape)
train_df = train_df.dropna()
print(train_df.shape)

(49992, 9)
(28423, 9)


In [30]:
stop_words = nltk.corpus.stopwords.words('english')

In [34]:
train_df['reviewText_lower'] = train_df['reviewText'].str.lower()

In [35]:
train_df['reviewText_lower_no_punctuation'] = train_df['reviewText_lower'].apply(lambda x: re.sub(r'[^\w\s]','', x))

In [36]:
train_df['nb_words'] = train_df['reviewText_lower_no_punctuation'].apply(lambda x: len(x.split()))

In [37]:
train_df['nb_stopwords'] = train_df['reviewText_lower_no_punctuation'].apply(lambda x: len([word for word in x.split() if word in stop_words]))

In [38]:
train_df['avg_word_length'] = train_df['reviewText_lower_no_punctuation'].apply(lambda x: np.mean([len(t) for t in x.split()]) if np.mean([len(t) for t in x.split()  if t not in stop_words]) > 0 else 0)

In [39]:
train_df.head()

,reviewerID,reviewerName,reviewText,summary,reviewTime,rates_count,helpful_count,rating,useful_criteria,label_aux,reviewText_lower,reviewText_lower_no_punctuation,nb_words,avg_word_length,nb_stopwords
0,A3UPFTGAWZ3G2R,David J. Loftus,"Jenkins, a history professor and Member of Par...","Quite readable, nicely done","12 6, 2001",40,37,4,0.925000,false,"jenkins, a history professor and member of par...",jenkins a history professor and member of parl...,256,5.656250,93
2,A1A77B6DQQH436,"crescamp ""esc""",I didn't read this. I purchased it for a gift...,10-minute life lessons for kids,"02 12, 2013",3,0,3,0.000000,false,i didn't read this. i purchased it for a gift...,i didnt read this i purchased it for a gift f...,25,3.600000,14
3,AEAF4MRYHJZI,"Angelia Menchan ""acvermen.blogspot.com""",Fierce Angels by Sheri Park reads like a disse...,So FIERCE,"03 24, 2010",9,9,4,1.000000,true,fierce angels by sheri park reads like a disse...,fierce angels by sheri park reads like a disse...,272,4.580882,123
4,A3B7KU72LGWFER,"Grifel ""Tea Time""",Clearly this author had two goals in mind: 1) ...,Drivel!,"06 21, 2003",19,13,1,0.684211,false,clearly this author had two goals in mind: 1) ...,clearly this author had two goals in mind 1 to...,103,4.466019,47
5,A3JD07VHDLT5FF,"isala ""Isabel and Lars""",This is a collection of stories and memories b...,Compelling stories by ordinary people,"03 19, 2005",7,5,5,0.714286,false,this is a collection of stories and memories b...,this is a collection of stories and memories b...,261,4.743295,134


In [40]:
train_df.shape

(28423, 15)